# Generate Training Data

We wish to speed up the process of having a human being name entities that they recognize in a paragraph, and having their positions in the paragraph identified and placed in a syntax usable by spaCy's NER training routine.

## Open PDF file, extract page two and display as sentences

In [1]:
import spacy
import PyPDF2
nlp = spacy.load('en_core_web_sm')

#Open PDF file for reading
pdfFile = open("BarCvDescLJ11.pdf", mode="rb")
pdfReader = PyPDF2.PdfFileReader(pdfFile)

# Select a page to work on
pageNumber = 13

# Get text
OnePage = pdfReader.getPage(pageNumber-1) #0-based count
OnePageText = OnePage.extractText()

# Close PDF file
pdfFile.close()

# Remove newlinesxmx82k-&a. It appears multiple newlines together makes
# Spacy think that is the end of a sentence. The PDF reader reads the text in
# an odd fashion
OnePageText = OnePageText.replace('\n','')

# create a spaCy doc object from the page and break it into sentences
doc = nlp(OnePageText)
l=0
for sent in doc.sents:
     print(l, ": ", sent)
     l = l+1


0 :  13    
1 :  BOB  
2 :  Bob is a two-rowed spring malting and feed barley.
3 :  It was released by the Washington State University Agriculture Research Center in cooperation with the Idaho and Oregon Agricultural Experiment Stations, and USDA-ARS in 2002.
4 :  It was selected from the cross A308 (Lewis somaclonal line)/Baronesse.
5 :  Its experimental designation was WA8682-96.  
6 :  It has midseason maturity, medium plant height (about 3 cm taller than Baronesse), fair straw strength, lax nodding spikes, rough long awns, and plump white kernels with long rachilla hairs.
7 :  It has mixed deficiens-wild type head types at a ratio of about 44%/56%.
8 :  It was widely adapted across eastern Washington and in general across Idaho and Oregon.
9 :  Yield averages 98% of Baronesse in eastern Washington.
10 :  Test weight and kernel plumpness average 90% and 85%, respectively, of Baronesse.  
11 :  Maturity is 1 day later than Baronesse.
12 :  Malting quality is comparable to Harrington,

## Read in Per-line named entity file and match entities to sentence positions.

In [3]:
import re
import csv
import pandas as pd
fname = "Data/barley_p13_ner.txt"

# Covert the nlp senetence generator into a list of sentences
sentences = list(doc.sents)

# Open the file of manually matched pairs (sentence # <tab> word phrase <tab> named entity)
# e.g.:
#  0      AC Metcalfe     CVAR
#  0      two-rowed       TRAT
#  0      barley          CROP
#  1      Agri-Food Candada   ORG
#  1      1997    DATE
file = open(fname)
reader = csv.reader(file, delimiter='\t', quoting=csv.QUOTE_NONE)
data = list()

for row in reader:
    try:
        (sentIndex, phrase, label) = row
        sent = sentences[int(sentIndex)].string.rstrip()
        
        # find all instances of the 'phrase' in the 'sent'.
        iter = re.finditer(r"\b"+phrase+r"\b", sent)
        indices = [m.start(0) for m in iter]
        
        # check to make sure the phrase the user said was there was indeed found
        if len(indices) == 0:
            raise ValueError
                
        # print out all instances
        for i in indices:
#            print(sentIndex, sent, phrase, "("+str(i), i+len(phrase), "'"+label+"')")
            data.append([sentIndex, sent, phrase, "("+str(i)+", "+str(i+len(phrase))+", '"+label+"')"])
            
    except:
        print("Handle manually: ", row)
        
df = pd.DataFrame(data, columns = ["Index", "Sentence", "Phrase", "MatchInfo"])
print(df)


Handle manually:  ['4', 'A308 (Lewis somaclonal line)/Baronesse', 'PED']
Handle manually:  ['15', 'Crop Science 43:1132-1133 (2003)', 'JRNL']
Handle manually:  ['19', 'Klages//Fergus/Nordic/3/ND1156/4/Hector', 'PED']
Handle manually:  ['29', 'Crop Science 25:883 (1985)', 'JRNL']
Handle manually:  ['46', 'Crop Science 8:776 (1968)', 'JRNL']
Handle manually:  ['62', 'Crop Science 42:666 (2002)', 'JRNL']
    Index                                           Sentence       Phrase  \
0       2  Bob is a two-rowed spring malting and feed bar...          Bob   
1       2  Bob is a two-rowed spring malting and feed bar...    two-rowed   
2       2  Bob is a two-rowed spring malting and feed bar...       spring   
3       2  Bob is a two-rowed spring malting and feed bar...      malting   
4       2  Bob is a two-rowed spring malting and feed bar...         feed   
..    ...                                                ...          ...   
154    68  At the time of evaluation it was moderately s

## Create a function to clean up overlapping intervals

In [4]:
import re
coordRegex = re.compile(r'(\d+), (\d+)')

def sortByStart(coords):
    """For use in sort routines, return object with lowest (X,Y) values"""
    # split out coordinates that come in as (5, 7, 'CVAR')
    mo = coordRegex.search(coords)
    return(int(mo.group(1)))

def overlaps(coord1, coord2):
    """Check if coordinates of the form 5, 7, 'CVAR' and 32, 46, 'TRAT' overlap"""
    mo1 = coordRegex.search(coord1)
    mo2 = coordRegex.search(coord2)
    coord1Low = int(mo1.group(1))
    coord1High = int(mo1.group(2))
    coord2Low = int(mo2.group(1))
    coord2High = int(mo2.group(2))
    
    if ((coord1High >= coord2Low) and (coord1Low <= coord2Low) or
        (coord2High >= coord1Low) and (coord2Low <= coord1Low)):
        return True
    else:
        return False

def keepFirst(coord1, coord2):
    """Given overlapping coordinates, return the wider encompassing one."""
    mo1 = coordRegex.search(coord1)
    mo2 = coordRegex.search(coord2)
    coord1Low = int(mo1.group(1))
    coord1High = int(mo1.group(2))
    coord2Low = int(mo2.group(1))
    coord2High = int(mo2.group(2))
 
    if (int(coord1High) - int(coord1Low)) >= (int(coord2High) - int(coord2Low)):
        return True
    else:
        return False

# print("Should be false:", overlaps("(5, 7, 'CVAR')", "(32, 46, 'TRAT')"))
# print("Should be true:", overlaps("(26, 46, 'TRAT')", "(32, 46, 'TRAT')"))
# print("Should be true:", overlaps("(26, 46, 'TRAT')", "(26, 46, 'TRAT')"))
# print("Keeper:", keepFirst("34, 46, 'TRAT'", "34, 46, 'TRAT'"))

def cleanIntervals(inputString=""):
    """order intervals like (5, 7, 'CVAR'), (32, 46, 'TRAT'), (26, 46, 'TRAT') and remove overlapping ones."""
    inputString = inputString.lstrip("(").rstrip(")")
    intervalList = inputString.split("), (")
    intervalList.sort(key = sortByStart)
#    print("Sorted Interval List:", intervalList)

    # Pairwise compare every interval in the list to every other interval to check overlap
    keeperList = [True]*len(intervalList) # Logic array to determine if each interval should be kept
    i=0
    for interval1 in intervalList:
        for interval2 in intervalList:
            if interval1 == interval2:
                if intervalList.index(interval1) != i: # when both are the same we reject the higher one
                    keeperList[i] = False
            else:
                if overlaps(interval1, interval2) and keepFirst(interval1, interval2) == False:
                    keeperList[i] = False
        i = i+1
        
#    print("keeperList:", keeperList)
   
    # Build up the return interval list
    returnStr = "("
    for interval, isKeeper in zip(intervalList, keeperList):
        if isKeeper:
            returnStr = returnStr + interval + "), ("
    return (returnStr.rstrip("), (") + ")")
        
# cleanIntervals("(5, 7, 'CVAR'), (32, 46, 'TRAT'), (5, 9, 'CVAR'), (48, 55, 'ORG'), (26, 46, 'TRAT')")
# cleanIntervals("(0, 8, 'CVAR'), (0, 5, 'CVAR'), (21, 26, 'PLAN'), (32, 37, 'CVAR')")
cleanIntervals("(0, 12, 'CVAR'), (39, 49, 'CVAR'), (39, 49, 'CVAR'), (71, 77, 'CVAR'), (71, 77, 'CVAR'), (92, 113, 'TRAT'), (140, 150, 'CVAR'), (140, 150, 'CVAR'), (181, 187, 'CVAR'), (181, 187, 'CVAR')")


"(0, 12, 'CVAR'), (39, 49, 'CVAR'), (71, 77, 'CVAR'), (92, 113, 'TRAT'), (140, 150, 'CVAR'), (181, 187, 'CVAR')"

## Aggregate all matches for each sentence on a single line and output in spaCy training format

In [5]:
# use Pandas dataframes to aggregate all entity matches together for a single sentence
agg_rules = {'Sentence': 'first', 'Phrase': 'first', 'MatchInfo': lambda x: ', '.join(x)}
res = df.groupby('Index').agg(agg_rules)
#print(res)

# Now format it just like what is needed for the spaCy training module: 
# E.g.:
# ('Eight-Twelve is a six-rowed winter feed barley', {'entities': [(0, 12, 'CVAR'), (18, 27, 'TRAT'), (28, 39, 'TRAT'),(40, 46, 'CROP')]}),
records = res.to_dict('records')
for i in range(0,len(records)):
    print("    ('"+records[i]['Sentence']+"', {'entities': ["+cleanIntervals(records[i]['MatchInfo'])+"]}),")
    #print("    ('"+records[i]['Sentence']+"', {'entities': ["+records[i]['MatchInfo']+"]}),")


    ('Test weight and kernel plumpness average 90% and 85%, respectively, of Baronesse.', {'entities': [(0, 11, 'TRAT'), (16, 22, 'PLAN'), (71, 80, 'CVAR')]}),
    ('Maturity is 1 day later than Baronesse.', {'entities': [(0, 8, 'TRAT'), (29, 38, 'CVAR')]}),
    ('Malting quality is comparable to Harrington, based on 4-year average.', {'entities': [(0, 15, 'TRAT'), (33, 43, 'CVAR')]}),
    ('At the time of release it was moderately resistant to leaf rust, moderately susceptible to stripe rust, and with unknown reaction to other diseases.', {'entities': [(30, 50, 'PPTD'), (54, 63, 'PATH'), (65, 87, 'PPTD'), (91, 102, 'PATH')]}),
    ('Bowman is a two-rowed spring feed barley.', {'entities': [(0, 6, 'CVAR'), (12, 21, 'TRAT'), (22, 28, 'TRAT'), (29, 33, 'TRAT'), (34, 40, 'CROP')]}),
    ('It was released by the North Dakota AES in cooperation with the USDA-ARS in 1984.', {'entities': [(23, 39, 'ORG'), (64, 72, 'ORG'), (76, 80, 'DATE')]}),
    ('Bob is a two-rowed spring malting and feed b